In [3]:
import pandas as pd
import plotly.express as px
import warnings 
from internproject.constants import ROOT_DATA_DIR

add_lp = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_add_lp.parquet")  # more details on the add liquidity events
remove_lp = pd.read_parquet(
    ROOT_DATA_DIR / "stETH_ETH_remove_lp.parquet"
)  # more details on the remove liquidity events
lp_transfers = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_lp_transfers.parquet")

In [4]:
stETH_ETH_apr_df = pd.read_csv(
    ROOT_DATA_DIR
    / 'daily_snapshots/0xDC24316b9AE028F1497c275EB9192a3Ea0f67022_apr_df.csv',
    index_col=0,
    parse_dates=True,
).dropna()

# finding rolled avg apr from og steth eth dataframe and set it as price 
stETH_ETH_apr_df['cur_naive_apr'] = stETH_ETH_apr_df[['raw_base_apr', 'raw_fee_apr', 'raw_incentive_apr', 'raw_price_apr']].sum(axis=1)
stETH_ETH_apr_df['rolling_cur_naive_apr'] = stETH_ETH_apr_df['cur_naive_apr'].rolling(7).mean()
stETH_ETH_apr_df['rolling_cur_naive_apr'].median()

5.086284024877622

### Reward rate data 

In [42]:
from internproject.constants import ROOT_DATA_DIR
stETH_ETH_incentive_rewards_df = pd.read_parquet(ROOT_DATA_DIR / 'raw_pool_data/curve_stETH_ETH/extra_rewards_df.parquet')
px.line(stETH_ETH_incentive_rewards_df['reward_rate'])
# this is the quanitty of LDO rewards issued per second.
# This can be used a replacement for APR that is not circular

In [44]:
px.line(stETH_ETH_incentive_rewards_df['reward_rate_eth'])

In [5]:
null_address = "0x0000000000000000000000000000000000000000"

### Understanding lp transfers

1. The "value" column in `lp_transfers` refers to the quantity of lp tokens
2. If the null address is in the "from" column, then it is adding liquidity to the pool. 
3. If the null address is in the "to" column, then it is removeing liquidity from the pool.
4. It is safe to ignore the other transfers

In [7]:
add_liquidity_transfers = lp_transfers[lp_transfers["from"] == null_address]
withdraw_liquidity_transfers = lp_transfers[lp_transfers["to"] == null_address]

In [8]:
withdraw_liquidity_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 17:12:33,0x3a63aA8AE11Ecaa82fC352d0390634E8030B2ffd,0x0000000000000000000000000000000000000000,2.000945,Transfer,11595786,133,295,0xbd58a23237562e0332aaa7a7668f8dd71e4aecddc95d...,2021-01-05 17:12:33
2021-01-05 17:46:38,0xaF297deC752c909092A117A932A8cA4AaaFF9795,0x0000000000000000000000000000000000000000,499.501433,Transfer,11595955,11,5,0xad790ed2cb537281ae0c7eaf1558b1494b869e7f7e1c...,2021-01-05 17:46:38
2021-01-05 21:03:54,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D,0x0000000000000000000000000000000000000000,0.999754,Transfer,11596894,75,96,0xc485ceb5703ddaf5393049b8fb71a06b20ee0c0b4ced...,2021-01-05 21:03:54
2021-01-06 03:36:23,0x89B8B20AE90328692cD367f75aaFadF55fd33E8B,0x0000000000000000000000000000000000000000,10.037649,Transfer,11598604,77,130,0xbebdcd6fa25430bea017919704fd3dcd12f55ef0f99b...,2021-01-06 03:36:23
2021-01-06 12:17:18,0xdE3E412d4fe3c9d90ac74d0A9B064951B39EEae4,0x0000000000000000000000000000000000000000,1.002055,Transfer,11600957,56,66,0x5a33f4f9e2e2a55f09141909f75b1b308d092183681c...,2021-01-06 12:17:18
...,...,...,...,...,...,...,...,...,...
2023-06-20 22:25:35,0xEa508F82728927454bd3ce853171b0e2705880D4,0x0000000000000000000000000000000000000000,0.000000,Transfer,17523858,45,95,0x6188d0177bb9a8a4a3bb5feb560477d6ec4642c9fde0...,2023-06-20 22:25:35
2023-06-20 22:33:11,0x46D5F548E02fdee3ea7E4168B7cF896429fE6fe7,0x0000000000000000000000000000000000000000,65.033669,Transfer,17523896,78,240,0x113260903e90d948871a8a0af2f03156c17781a455ea...,2023-06-20 22:33:11
2023-06-20 22:40:35,0x5aD15EBa9D4E9351414084Eb1dd1EACc2068A4A7,0x0000000000000000000000000000000000000000,51.167100,Transfer,17523933,42,101,0x39af71a3003ae573c6739b9679bede548ea7cc53a6b0...,2023-06-20 22:40:35


### Analyzing behavior and liquidity of unique users 
- combining the added and withdrawed liquidty 

In [11]:
withdraw_liquidity_transfers['value'] = withdraw_liquidity_transfers['value']* -1

warnings.filterwarnings('ignore')
withdraw_liquidity_transfers['value']

timestamp
2021-01-05 17:12:33     -2.000945
2021-01-05 17:46:38   -499.501433
2021-01-05 21:03:54     -0.999754
2021-01-06 03:36:23    -10.037649
2021-01-06 12:17:18     -1.002055
                          ...    
2023-06-20 22:25:35     -0.000000
2023-06-20 22:33:11    -65.033669
2023-06-20 22:40:35    -51.167100
2023-06-20 23:28:11     -0.000000
2023-06-21 00:00:11     -0.003230
Name: value, Length: 27851, dtype: float64

In [12]:
add_liquidity_transfers['user'] = add_liquidity_transfers['to']
withdraw_liquidity_transfers['user'] = withdraw_liquidity_transfers['from']

transfer_df = pd.concat([add_liquidity_transfers, withdraw_liquidity_transfers])
transfer_df.columns

warnings.filterwarnings('ignore')

In [13]:
# locating one unique user
transfer_df['user'].value_counts().head(100).tail(1)

0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D    25
Name: user, dtype: int64

In [54]:
transfer_df['user'] == '0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7'

timestamp
2021-01-05 00:26:18    False
2021-01-05 11:26:21    False
2021-01-05 13:35:18     True
2021-01-05 13:48:47     True
2021-01-05 14:32:47    False
                       ...  
2023-06-20 22:31:59    False
2023-06-20 22:33:11    False
2023-06-20 22:40:35    False
2023-06-20 23:28:11    False
2023-06-21 00:00:11    False
Name: user, Length: 62068, dtype: bool

In [47]:
transfer_df_1person = transfer_df[transfer_df['user'] == '0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7']

In [61]:
transfer_df_1person

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col,balance_over_time
timestamp,,,,,,,,,,,,
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,69.999
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,2069.989490,2069.989
2021-01-13 13:27:28,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1790.364658,Transfer,11646866,51,89,0xce8992a8ad1e9f650ed78f8d387f507ce0d420719776...,2021-01-13 13:27:28,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3860.354147,3860.354
2021-02-01 22:02:17,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-199.860373,Transfer,11772899,44,63,0xef7c06a31d0fb37afe80300cbbb25b0b51f71a8ab162...,2021-02-01 22:02:17,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3660.493775,3660.494
2021-03-08 14:49:11,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-3660.493775,Transfer,11998460,184,170,0xd21e15bceedcc71042088f39a39ab4bffd5d3f750351...,2021-03-08 14:49:11,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0.000000,0.000
2021-04-10 19:41:29,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3718.426815,Transfer,12214131,160,323,0xa2d93409e7514d01c04231d17b5248ab5e0947dcdfff...,2021-04-10 19:41:29,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3718.426815,3718.427
2022-12-24 16:04:23,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-3718.426815,Transfer,16255668,11,62,0xd6bef2e8d1fef6f333730b95e9b1c0de5da4a656c242...,2022-12-24 16:04:23,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0.000000,0.000


In [62]:
# finding cumulative sum of this person's transfer payments, which is the same as their balance over time
transfer_df_1person['value'].cumsum().round(3).values
transfer_df_1person['balance_over_time'] = transfer_df_1person['value'].cumsum().round(3).values

# merge reward rate df with this df so we can access the reward rates for this whale's transfers
transfer_df_1person_reward = pd.merge(transfer_df_1person, stETH_ETH_incentive_rewards_df, left_on=transfer_df_1person.index, right_on=stETH_ETH_incentive_rewards_df.index, how='left')

warnings.filterwarnings('ignore')

In [63]:
transfer_df_1person_reward

,key_0,from,to,value,event,block_x,transaction_index,log_index,hash,timestamp,user,cumsol_col,balance_over_time,lp_token_total_staked,reward_rate,block_y,reward_rate_eth
0,2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,69.999,NaN,NaN,NaN,NaN
1,2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,2069.989490,2069.989,NaN,NaN,NaN,NaN
2,2021-01-13 13:27:28,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1790.364658,Transfer,11646866,51,89,0xce8992a8ad1e9f650ed78f8d387f507ce0d420719776...,2021-01-13 13:27:28,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3860.354147,3860.354,NaN,NaN,NaN,NaN
3,2021-02-01 22:02:17,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-199.860373,Transfer,11772899,44,63,0xef7c06a31d0fb37afe80300cbbb25b0b51f71a8ab162...,2021-02-01 22:02:17,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3660.493775,3660.494,NaN,NaN,NaN,NaN
4,2021-03-08 14:49:11,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-3660.493775,Transfer,11998460,184,170,0xd21e15bceedcc71042088f39a39ab4bffd5d3f750351...,2021-03-08 14:49:11,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0.000000,0.000,NaN,NaN,NaN,NaN
5,2021-04-10 19:41:29,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3718.426815,Transfer,12214131,160,323,0xa2d93409e7514d01c04231d17b5248ab5e0947dcdfff...,2021-04-10 19:41:29,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3718.426815,3718.427,NaN,NaN,NaN,NaN
6,2022-12-24 16:04:23,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-3718.426815,Transfer,16255668,11,62,0xd6bef2e8d1fef6f333730b95e9b1c0de5da4a656c242...,2022-12-24 16:04:23,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0.000000,0.000,NaN,NaN,NaN,NaN


In [65]:
# amount of tokens supplied (balance) over time 
px.bar(transfer_df_1person_reward[['balance_over_time', 'reward_rate_eth']])

### WHALES!!

In [18]:
# grouping df by user and calculating each individual's cumulative sum to FIND WHALES 
transfer_df.sort_index(inplace=True)
transfer_df['cumsol_col'] = transfer_df.groupby(['user'])['value'].cumsum()

# create pivot table that contains unique users and their cumulative sum 
pivot_df = transfer_df.pivot_table(values='cumsol_col', index=transfer_df.index, columns='user')

In [56]:
# locate whale by finding where at one point user had over a certain amount
whale_df = transfer_df.loc[transfer_df['cumsol_col'] > 500]

# find how many unique whales there are 
num_unique_whales = whale_df.groupby(['user']) 
len(num_unique_whales.groups)
# there are 2157 unique whales 

785

In [40]:
# there are 16,152 unique users in the OVERALL pool 
grouped = transfer_df.groupby(['user'])
print(grouped.groups)


{'0x0000000000117D8C0c6103ecA077462224723192': [2022-09-17 07:09:47], '0x000000000037D42ab4e2226CE6f44C5dC0Cf5b16': [2022-05-15 00:02:54, 2022-05-15 08:30:59, 2022-05-15 14:08:02, 2022-05-16 01:18:22, 2022-05-28 22:05:38], '0x00000000245A821a3e646b8187636AD09Cd9cCB0': [2022-10-16 19:37:59], '0x000000003fBC5761A6849dC46d6C292c7B01e894': [2023-06-01 10:00:59], '0x0000000090E2bA937F0b5615b57409a8A6a1D6Dd': [2021-08-15 14:08:51, 2021-08-15 14:47:39], '0x0000005D07ee3043DAba2bc7dbfEa2d9b048b985': [2022-08-16 22:08:26, 2022-08-25 22:17:38], '0x000000849994057de6A5E789F1e60043Ae8d562D': [2022-08-26 05:48:21, 2022-09-01 11:23:47], '0x00000088f474D54EcA2F9c95d69Ac02b2660B0D5': [2023-05-06 01:42:11], '0x0000CE08fa224696A819877070BF378e8B131ACF': [2021-06-30 07:57:19, 2021-08-03 08:31:22], '0x0001196FCcC127952FC5a6f3a5D5546C7487F78a': [2021-12-08 14:50:45], '0x00029d35CB7aE09D38037355a046791D7b5E1645': [2021-10-09 00:33:19, 2022-04-01 17:30:45], '0x000566B53E028d21e104E4320dE61c2314ef4064': [2022

In [41]:
whale_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col
timestamp,,,,,,,,,,,
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,2069.989490
2021-01-05 14:41:13,0x0000000000000000000000000000000000000000,0x3DbA737ccC50a32a1764b493285dd51C8Af6c278,4720.978933,Transfer,11595139,130,168,0x8ec9b7ca0c5434f9bb9917f135b3d34f7841022913ba...,2021-01-05 14:41:13,0x3DbA737ccC50a32a1764b493285dd51C8Af6c278,4720.978933
2021-01-05 14:57:05,0x0000000000000000000000000000000000000000,0x97960149fc611508748dE01202974d372a677632,4001.568744,Transfer,11595201,60,124,0x0a4bdeec43ee0e170b6b46528dbe3c43809d72962d0e...,2021-01-05 14:57:05,0x97960149fc611508748dE01202974d372a677632,4001.568744
2021-01-05 16:59:23,0x0000000000000000000000000000000000000000,0x735f4775594BA291C1A79C36D4A99889143600e3,149.703061,Transfer,11595721,94,208,0xee8cc946577299058eb03062004b4c4237448c3c68b5...,2021-01-05 16:59:23,0x735f4775594BA291C1A79C36D4A99889143600e3,149.703061
2021-01-05 17:29:36,0x0000000000000000000000000000000000000000,0xaF297deC752c909092A117A932A8cA4AaaFF9795,499.501433,Transfer,11595874,34,16,0xd40d266799b5b622b4a0dbf5eb6858ac8d356c0235be...,2021-01-05 17:29:36,0xaF297deC752c909092A117A932A8cA4AaaFF9795,499.501433
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-20 02:10:47,0x0000000000000000000000000000000000000000,0xEf0D72C594b28252BF7Ea2bfbF098792430815b1,112.867796,Transfer,17517853,100,165,0xd412e88b3bc0d835075e43e4ac5742121e7a3d41f333...,2023-06-20 02:10:47,0xEf0D72C594b28252BF7Ea2bfbF098792430815b1,485.643304
2023-06-20 07:15:23,0x0000000000000000000000000000000000000000,0x682bA005aF4276749A26Dc18400b395DaC408191,46.516499,Transfer,17519363,95,190,0x4b3718a07e1a63e8c90c92d38bc40dbd53af21c8927d...,2023-06-20 07:15:23,0x682bA005aF4276749A26Dc18400b395DaC408191,297.309595
2023-06-20 16:15:59,0x597568694FE6eE1b701EC8578bd57102C9A29abd,0x0000000000000000000000000000000000000000,-7134.571638,Transfer,17522035,60,129,0x7fd2c7279901ce52e7694824046ea940d8b4db96271b...,2023-06-20 16:15:59,0x597568694FE6eE1b701EC8578bd57102C9A29abd,286.207120


In [51]:
px.line(whale_df['cumsol_col'])

In [60]:
pivot_df2 = transfer_df.pivot_table(values='cumsol_col', index='user')
pivot_df2 


,cumsol_col
user,
0x0000000000117D8C0c6103ecA077462224723192,-0.308512
0x000000000037D42ab4e2226CE6f44C5dC0Cf5b16,-37.188278
0x00000000245A821a3e646b8187636AD09Cd9cCB0,-85.109258
0x000000003fBC5761A6849dC46d6C292c7B01e894,-93.203660
0x0000000090E2bA937F0b5615b57409a8A6a1D6Dd,0.963734
...,...
0xff879CA9C84D976dB7E0180436e3a689eD70E1D2,0.000096
0xff8a51f0dE0166672a2f77f0213e6D814c28E36C,-36.296353
0xff9A717FCF03318485579c4b553820eDf73aDf0A,25.244343


In [14]:
pivot_df = pivot_df.ffill()

In [70]:
px.line(pivot_df['0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D'])

NameError: name 'pivot_df' is not defined

### when did this person enter and leave

In [6]:
example_person = "0x80ee7c0e1e59929823eD9B22e34538b226967109"
add_liquidity_transfers[add_liquidity_transfers["to"] == example_person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-12 22:16:32,0x0000000000000000000000000000000000000000,0x80ee7c0e1e59929823eD9B22e34538b226967109,1.999396,Transfer,11642841,30,51,0xb03636820959b9641a5c4c39676967180d7629a9bc30...,2021-01-12 22:16:32
2021-01-12 22:31:11,0x0000000000000000000000000000000000000000,0x80ee7c0e1e59929823eD9B22e34538b226967109,0.999698,Transfer,11642902,120,187,0xcf8723e0451638d3f68a12395facc8c0f1ed2178a90f...,2021-01-12 22:31:11


In [7]:
withdraw_liquidity_transfers[withdraw_liquidity_transfers["from"] == example_person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-14 13:13:07,0x80ee7c0e1e59929823eD9B22e34538b226967109,0x0000000000000000000000000000000000000000,2.999095,Transfer,11653414,210,243,0x642a0a0273035551b6fa585c8615e129bc05772c9277...,2021-01-14 13:13:07


This person entered twice at 2021-01-12 and then left at 2021-01-14. So they were only in for 3 days.

### Open Problems

Here are some problems you could look at. Remember that the focus is figuring out how changing incentive APR affacts LPs entering and leaving the pool

1. How does the amount of ETH value added influence the duration of a person's stay in the pool? (does this relationship change over time?)
2. Among those who fully exited the pool, what was the difference between the starting and ending Annual Percentage Rate (APR)?
3. What was the average duration of people's participation in the pool?
4. How frequently did people perform actions such as adding liquidity or withdrawing liquidity? Did they break up these actions into smaller increments?
5. Can the participants be clustered into meaningful groups based on their LP sizes and durations?
6. Are there any notable thresholds or breakpoints at which people exhibit different behaviors? eg. after a person has been in a pool for > 1 month they almost never leave
7. It is important to pay special attention to participants with large LPs. How do the people who have added and withdrawn large amounts of liquidity differ?

# Elasticity Subgroups

##### LP's who provided over 100 ETH 

In [207]:
# merge add_lp with stETH_ETH_apr_df so that we can access the apr's 
add_lp['date'] = pd.to_datetime(add_lp['timestamp'].dt.date)
merged_df = pd.merge(add_lp, stETH_ETH_apr_df, left_on='date', right_on='timestamp', how='left') 
merged_df.set_index('date', inplace=True)

In [277]:
# sum up tokens provided for each coin and select lp's that provided over 100 
merged_df['total_tokens'] = merged_df['token_amounts_0'] + merged_df['token_amounts_1']  
add_over_100_subgroup = merged_df.loc[merged_df['total_tokens'] > 100] 

subgroup_shift = add_over_100_subgroup.shift(-30, freq='D')
add_over_100_subgroup = pd.merge_asof(add_over_100_subgroup, subgroup_shift[['rolling_cur_naive_apr', 'token_supply']], left_index=True, right_index=True, direction='forward', suffixes=('', '_prev'))

In [279]:
# set variables for elasticity formula
q2 = add_over_100_subgroup['token_supply']
q1 = add_over_100_subgroup['token_supply_prev']
p2 = add_over_100_subgroup['rolling_cur_naive_apr']
p1 = add_over_100_subgroup['rolling_cur_naive_apr_prev']

# calculate elasticity 
percent_change_supply = (q2 - q1) / ((q2 + q1) / 2) * 100 
percent_change_price = (p2 - p1) / ((p2 + p1) / 2) * 100 
add_over_100_subgroup['elasticity'] = percent_change_supply / percent_change_price 
add_over_100_subgroup['elasticity']

warnings.filterwarnings('ignore')

##### Mostly similar, but we're missing the large negative shock in May 2023 AND the positive shock is much larger 
- As apr increases, we expect quantity to increase, i.e. all elasticity values should be positive. This market seems to behave more rationally than the other market. 

In [308]:
px.line(add_over_100_subgroup['elasticity'].dropna())

In [311]:
px.line(add_over_100_subgroup['market_eth_tvl'].dropna())

- In regards to the large positive shock, there could be outside factors unrelated to apr that cause ppl to leave/enter. For example, crazy low apr in other markets would lead to lots of people entering stETH ETH market because they're attracted to the higher return. 
- other markets unsafe

#### LP's that added liquidity when APR is greater than median percentage

In [288]:
# convert to datetime so we don't lose data, then merge add_liquidity_transfers with stETH_ETH_apr_df SO WE CAN ACCESS APR'S 
add_liquidity_transfers['date_only'] = pd.to_datetime(add_liquidity_transfers.index.date)
merge2_df = pd.merge(add_liquidity_transfers, stETH_ETH_apr_df, left_on='date_only', right_on='timestamp', how='left')
merge2_df.set_index('date_only', inplace=True)


In [300]:
# find where lp's added liquidity when apr is greater than median 
added_when_apr_over_med = merge2_df.loc[merge2_df['rolling_cur_naive_apr'] > merge2_df['rolling_cur_naive_apr'].median()]
# added_when_apr_over_med = added_when_apr_over_med[~added_when_apr_over_med.index.duplicated(keep='first')]

shift = added_when_apr_over_med.shift(-30, freq='D')
added_when_apr_over_med = pd.merge_asof(added_when_apr_over_med, shift[['rolling_cur_naive_apr', 'value']], left_index=True, right_index=True, direction='forward', suffixes=('', '_prev'))


In [301]:
# set price variables 
p2 = added_when_apr_over_med['rolling_cur_naive_apr']
p1 = added_when_apr_over_med['rolling_cur_naive_apr_prev']

# set supply variables
q2 = added_when_apr_over_med['value']
q1 = added_when_apr_over_med['value_prev']

# calculate elasticity
percent_change_supply = (q2 - q1) / ((q2 + q1) / 2) * 100 
percent_change_price = (p2 - p1) / ((p2 + p1) / 2) * 100 
added_when_apr_over_med['elasticity'] = percent_change_supply / percent_change_price


In [305]:
px.line(added_when_apr_over_med['elasticity'].dropna())

######
This elasticity does not seem to even be influenced by apr. The infinitely large elasticity values could indicate perfect elasticity or error in the code 

In [303]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=added_when_apr_over_med.index, y=added_when_apr_over_med['elasticity'], name='elasticity'), 
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=added_when_apr_over_med.index, y=added_when_apr_over_med['rolling_cur_naive_apr'], name='price (apr)'), 
    secondary_y=True,
)

# Set x-axis title
fig.update_xaxes(title_text="time stamp")

# Set y-axes titles
fig.update_yaxes(title_text="<b>elasticity</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>price (measured in apr)</b>", secondary_y=True)